In [1]:
from ltn_imp.automation.knowledge_base import KnowledgeBase
import torch
import pandas as pd

In [2]:
!poetry run poe download-medical-datasets

Poe => mkdir -p examples/medical/datasets
Poe => curl -L -o examples/medical/datasets/pima_indians_imputed.csv https://raw.githubusercontent.com/ChristelSirocchi/hybrid-ML/main/pima_indians_imputed.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 33428  100 33428    0     0   561k      0 --:--:-- --:--:-- --:--:--  562k


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import random
import numpy as np

seed = 42

# Set the random seed for reproducibility
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [5]:
from sklearn.model_selection import train_test_split

test_data = pd.read_csv('datasets/pima_indians_imputed.csv', index_col=0).astype(float)

y = test_data.iloc[:, -1]

x_train, x_test = train_test_split(test_data, test_size=0.5, random_state=seed, stratify=y)
y_train = x_train.iloc[:, -1]  # Extract labels from the training split
x_train, x_val = train_test_split(x_train, test_size=0.1, random_state=seed, stratify=y_train)

x_train.to_csv('datasets/train.csv')
x_val.to_csv('datasets/val.csv')
x_test.to_csv('datasets/test.csv')

In [6]:
import torch
from sklearn.metrics import matthews_corrcoef

def predict(model, x):
    model.eval()  # Ensure the model is in evaluation mode
    with torch.no_grad():  # No need to track gradients
        # Ensure x is a tensor and has the right dtype
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)
        elif x.dtype != torch.float32:
            x = x.float()
        
        # Forward pass through the model
        probs = model(x)
        
        # Apply binary classification threshold at 0.5
        preds = (probs > 0.5).float()
    return preds

def compute_metrics(model, data_loader):
    correct = 0
    total = 0
    
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    true_negatives = 0
    
    all_true_labels = []
    all_predicted_labels = []
    
    with torch.no_grad():  # Disable gradient computation
        for data, labels in data_loader:
            # Ensure data and labels are the correct dtype
            if not isinstance(data, torch.Tensor):
                data = torch.tensor(data, dtype=torch.float32)
            elif data.dtype != torch.float32:
                data = data.float()
            
            if not isinstance(labels, torch.Tensor):
                labels = torch.tensor(labels, dtype=torch.float32)
            elif labels.dtype != torch.float32:
                labels = labels.float()
            
            # Get predictions
            preds = predict(model, data)
            
            # Squeeze predictions and labels to remove dimensions of size 1
            predicted_labels = preds.squeeze()
            true_labels = labels.squeeze()

            # Ensure the shapes match before comparison
            if predicted_labels.shape != true_labels.shape:
                true_labels = true_labels.view_as(predicted_labels)
            
            # Collect all predictions and true labels for MCC
            all_true_labels.extend(true_labels.cpu().numpy())
            all_predicted_labels.extend(predicted_labels.cpu().numpy())

            # Count correct predictions
            correct += (predicted_labels == true_labels).sum().item()
            total += true_labels.size(0)
            
            # Calculate TP, FP, FN, TN
            true_positives += ((predicted_labels == 1) & (true_labels == 1)).sum().item()
            false_positives += ((predicted_labels == 1) & (true_labels == 0)).sum().item()
            false_negatives += ((predicted_labels == 0) & (true_labels == 1)).sum().item()
            true_negatives += ((predicted_labels == 0) & (true_labels == 0)).sum().item()
    
    accuracy = correct / total if total > 0 else 0
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    balanced_accuracy = 0.5 * (recall + (true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0))
    mcc = matthews_corrcoef(all_true_labels, all_predicted_labels)
    tnr = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0  # True Negative Rate
    fpr = false_positives / (false_positives + true_negatives) if (false_positives + true_negatives) > 0 else 0  # False Positive Rate
    fnr = false_negatives / (false_negatives + true_positives) if (false_negatives + true_positives) > 0 else 0  # False Negative Rate
    tpr = recall  # True Positive Rate is the same as recall

    print(f"True Positives: {true_positives}, False Positives: {false_positives}, False Negatives: {false_negatives}, True Negatives: {true_negatives}, Total: {total}")
    print()
    print(f"A (Accuracy): {accuracy:.4f}")
    print(f"P (Precision): {precision:.4f}")
    print(f"R (Recall): {recall:.4f}")
    print(f"F1 (F1 Score): {f1_score:.4f}")
    print(f"BA (Balanced Accuracy): {balanced_accuracy:.4f}")
    print(f"MCC (Matthews Correlation Coefficient): {mcc:.4f}")
    print(f"TNR (True Negative Rate): {tnr:.4f}")
    print(f"FPR (False Positive Rate): {fpr:.4f}")
    print(f"FNR (False Negative Rate): {fnr:.4f}")
    print(f"TPR (True Positive Rate): {tpr:.4f}")
    model.train()

In [7]:
kb = KnowledgeBase("with_logic.yaml", device="cpu")

Using device: cpu


In [8]:
fine_tune = True

if fine_tune:
    model = kb.predicates["Diabetic"]

    criteria = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)

    patience = 5
    min_delta = 0.0
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(50):
        model.train()
        total_loss = 0.0
        num_batches = 0
        
        # Training loop
        for data, labels in kb.loaders[0]:
            optimizer.zero_grad()
            predictions = model(data)
            loss = criteria(predictions, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            num_batches += 1

        avg_train_loss = total_loss / num_batches

        model.eval()
        total_val_loss = 0.0
        num_val_batches = 0

        for data, labels in kb.val_loaders[0]:
            with torch.no_grad():
                predictions = model(data)
                val_loss = criteria(predictions, labels)
                total_val_loss += val_loss.item()
                num_val_batches += 1

        avg_val_loss = total_val_loss / num_val_batches

        # Early stopping logic
        if avg_val_loss + min_delta < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"Early Stopping at: Epoch {epoch + 1}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")
            break

        if epoch % 10 == 0:
            print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

        model.train()

Epoch 1, Train Loss: 0.655768, Val Loss: 0.664067
Epoch 11, Train Loss: 0.583104, Val Loss: 0.436683
Early Stopping at: Epoch 18, Train Loss: 0.469982, Val Loss: 0.489238


In [9]:
compute_metrics(kb.predicates["Diabetic"], kb.loaders[0])

True Positives: 93, False Positives: 60, False Negatives: 27, True Negatives: 165, Total: 345

A (Accuracy): 0.7478
P (Precision): 0.6078
R (Recall): 0.7750
F1 (F1 Score): 0.6813
BA (Balanced Accuracy): 0.7542
MCC (Matthews Correlation Coefficient): 0.4873
TNR (True Negative Rate): 0.7333
FPR (False Positive Rate): 0.2667
FNR (False Negative Rate): 0.2250
TPR (True Positive Rate): 0.7750


In [10]:
if fine_tune:
    kb.optimize(num_epochs=100, log_steps=10, lr=0.0000001, early_stopping=True, patience=5)
else:
    kb.optimize(num_epochs=150, log_steps=50, lr=0.00001, early_stopping=True, patience=5)

['∀ person.(((y == diabetes) -> Diabetic(person)))', '∀ person.(((y == healthy) -> ~(Diabetic(person))))', '∀ person.((((person[BMI] > 29) & (person[Glucose] > 125)) -> Diabetic(person)))', '∀ person.((((person[BMI] < 26) & (person[Glucose] < 101)) -> ~(Diabetic(person))))', '∀ person.((((person[Glucose] > 143.5) & (person[DiabetesPedigreeFunction] > 0.32)) -> Diabetic(person)))', '∀ person.((((person[Glucose] > 143.5) & ((person[DiabetesPedigreeFunction] <= 0.32) & (person[BMI] > 31.40))) -> Diabetic(person)))', '∀ person.((((person[Glucose] > 143.5) & ((person[DiabetesPedigreeFunction] <= 0.32) & (person[BMI] <= 31.40))) -> ~(Diabetic(person))))', '∀ person.((((person[Glucose] <= 143.5) & ((person[Pregnancies] > 7.5) & (person[DiabetesPedigreeFunction] > 0.50))) -> Diabetic(person)))', '∀ person.((((person[Glucose] <= 143.5) & ((person[Pregnancies] <= 7.5) & (person[BMI] > 45.44))) -> Diabetic(person)))', '∀ person.((((person[Glucose] <= 143.5) & ((person[Pregnancies] <= 7.5) & (pers

In [11]:
compute_metrics(kb.predicates["Diabetic"], kb.loaders[0])

True Positives: 89, False Positives: 61, False Negatives: 31, True Negatives: 164, Total: 345

A (Accuracy): 0.7333
P (Precision): 0.5933
R (Recall): 0.7417
F1 (F1 Score): 0.6593
BA (Balanced Accuracy): 0.7353
MCC (Matthews Correlation Coefficient): 0.4521
TNR (True Negative Rate): 0.7289
FPR (False Positive Rate): 0.2711
FNR (False Negative Rate): 0.2583
TPR (True Positive Rate): 0.7417


In [ ]:
compute_metrics(kb.predicates["Diabetic"], kb.loaders[0])

# A

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

x = test_data.iloc[:, :-1]

lr = LogisticRegression(random_state=seed)
lr.fit(x,y)
lr.score(x, y)

0.7786458333333334

In [13]:
mlp = MLPClassifier(random_state=seed, max_iter=10)
mlp.fit(x,y)
mlp.score(x, y)

0.6927083333333334

In [14]:
import torch.nn.functional as F
from torch import optim

class InPlaceParameter(torch.nn.Parameter):
    def __new__(cls, data, model_ref, model_attr):
        instance = super().__new__(cls, data, requires_grad=True)
        instance.model_ref = model_ref
        instance.model_attr = model_attr
        return instance

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        self.update_model_param()

    def __setattr__(self, name, value):
        super().__setattr__(name, value)
        if name in ['data', 'grad', 'requires_grad']:
            self.update_model_param()

    def update_model_param(self):
        setattr(self.model_ref, self.model_attr, self.detach().numpy())

def predict(X, coef, intercept):
    logits = X @ coef.T + intercept  
    probs = torch.sigmoid(logits)    
    return probs

def custom_loss(X, y, coef, intercept):
    probs = predict(X, coef, intercept)
    bce_loss = F.binary_cross_entropy(probs.squeeze(), y)
    return bce_loss


lr = LogisticRegression(random_state=seed, max_iter=1)
lr.fit(x,y)

model = lr

coef = InPlaceParameter(torch.tensor(model.coef_, dtype=torch.float32), model, 'coef_')
intercept = InPlaceParameter(torch.tensor(model.intercept_, dtype=torch.float32), model, 'intercept_')

optimizer = optim.Adam([coef, intercept], lr=0.02)
X_tensor = torch.tensor(x.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

for epoch in range(1001):
    optimizer.zero_grad()
    loss = custom_loss(X_tensor, y_tensor, coef, intercept)
    loss.backward()
    optimizer.step()

    if epoch % 250 == 0:
        print(f"Step {epoch + 1}/1000")
        print(f"Loss: {loss.item()}")
        print()

Step 1/1000
Loss: 0.6884763836860657

Step 251/1000
Loss: 0.5577901005744934

Step 501/1000
Loss: 0.5307449698448181

Step 751/1000
Loss: 0.5070579648017883

Step 1001/1000
Loss: 0.48979267477989197



In [15]:
model.score(x, y)

0.76171875